In [ ]:
import os
import re
import glob
import zipfile
import fnmatch
import datetime
from zipfile import ZipFile
import pandas as pd
from pandas import DataFrame

In [ ]:
def find_sheet(sheet_list, sheet_name):
    return [x for x in sheet_list if re.search(sheet_name.lower(), x.lower())]

In [ ]:
def create_timestamp():

    today = datetime.date.today()
    year = today.year
    month = today.month
    day = today.day
    
    timestamp = str(year)+str(month)+str(day)
    
    return timestamp

In [ ]:
data_dir = ".\\excel\\data\\"
archive_dir = ".\\excel\\archive\\"

if not os.path.exists(archive_dir):
    os.makedirs(archive_dir)
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)
    # exit() Use in stand-alone script.

In [ ]:
for zip_file in os.listdir(data_dir):
    with ZipFile(data_dir + zip_file, "r") as zip_obj:
        zip_obj.extractall(data_dir)
    os.remove(data_dir + zip_file)

In [ ]:
# Version 1

file_list = glob.glob(data_dir + "*.xlsx")

for xl_file in file_list:
    sheets = pd.ExcelFile(xl_file).sheet_names
    workbook = pd.ExcelFile(xl_file)

    if fnmatch.fnmatch(xl_file.lower(), "*grocery*.xlsx") == True:        
        df_grocery = workbook.parse(0)
        df_frozen = workbook.parse(1)
        df_dairy = workbook.parse(2)
#         df_grocery = pd.read_excel(xl_file, find_sheet(sheets, "grocery"))
#         df_frozen = pd.read_excel(xl_file, find_sheet(sheets, "frozen"))
#         df_dairy = pd.read_excel(xl_file, find_sheet(sheets, "dairy"))
    elif fnmatch.fnmatch(xl_file.lower(), "*hbc*.xlsx") == True:
        df_hbc = workbook.parse(0)
    elif fnmatch.fnmatch(xl_file.lower(), "*perish*.xlsx") == True:
        df_perish = workbook.parse(0)
        
cs_list = [df_grocery, df_frozen, df_dairy, df_hbc, df_perish]
df_cs_data = pd.concat(cs_list)
df_cs_data

In [ ]:
# Version 2

xl_list = glob.glob(data_dir + "*.xlsx")
cs_list = []

for xl_file in xl_list:
    sheet_list = pd.ExcelFile(xl_file).sheet_names
    workbook = pd.ExcelFile(xl_file)
    
    for (index, sheet) in enumerate(workbook.sheet_names):
        cs_list.append(workbook.parse(index))    
#     cs_list = [workbook.parse(x) for x, y in enumerate(workbook.sheet_names)]
df_cs_data = pd.concat(cs_list)
df_cs_data

In [ ]:
# Use code later to clean file

# val = df_grocery.iloc[:, [0, 1, 2, 3, 4, 5, 22, 23, 6, 7, 8, 9, 10, 11, 12]]
# val

In [ ]:
df_cs_data.to_excel(f"{archive_dir}CS-Sales-Change-{create_timestamp()}.xlsx")